In [2]:
import numpy as np
import pandas as pd

def load_data(file_path):
    # Charger les données : [temps, prix]
    data = pd.read_csv(file_path, skiprows=4)
    
    data.columns = ["Date", "Price"]
    
    # Convertir la colonne "Date" en datetime64 de NumPy
    data['Date'] = pd.to_datetime(data['Date'], format="%m/%d/%Y").values.astype('datetime64[D]')
    
    data = data.sort_values(by="Date")
    
    return data.to_numpy()

def select_sample(data, time_start, time_end):
    # Convertir time_start et time_end en datetime64
    time_start = np.datetime64(pd.to_datetime(time_start, format="%m/%d/%Y"))
    time_end = np.datetime64(pd.to_datetime(time_end, format="%m/%d/%Y"))
    
    # Sélectionne l’échantillon principal
    mask = (data[:,0] >= time_start) & (data[:,0] <= time_end)

    sample_data = data[mask]
    sample_data[:,0] = np.linspace(1, len(sample_data), len(sample_data))


    return sample_data.astype(float)

def generate_subintervals(data):
    """
    Génère les sous-intervalles suivant la logique du pseudo-code.
    - On calcule delta comme le max entre ((time_end - time_start)*0.75/21 jours) et 21 jours.
    - subinterval_end varie de time_end à time_end - 6 semaines (42 jours) par pas de 7 jours.
    - subinterval_start varie de time_start à time_end-(time_end-time_start)/4 par pas de delta.
    """

    time_start = data[0,0]
    time_end = data[-1,0]

    three_weeks = 21.0  # jours
    six_weeks = 42.0
    one_week = 7.0
    total_days = (time_end - time_start)
    delta = max((total_days * 0.75) / three_weeks, three_weeks)

    subintervals = []
    # Pour simplifier, on considère data[:,0] en jours continus
    for sub_end in np.arange(time_end, time_end - six_weeks, -one_week):
        for sub_start in np.arange(time_start, time_end - total_days/4, delta):
            mask = (data[:,0] >= sub_start) & (data[:,0] <= sub_end)
            sub_data = data[mask]            
            if len(sub_data) > 0:
                subintervals.append((sub_start, sub_end, sub_data))
    return subintervals




In [169]:
from numba import njit

@njit
def initialize_population(param_bounds, population_size):
    """
    Initialize a population in nopython mode with Numba.
    
    Parameters
    ----------
    param_bounds : 2D ndarray of shape (D, 2), float64
        Rows correspond to each parameter [low, high].
        For example, if we have 4 parameters:
            param_bounds[0] = [t_c_min, t_c_max]
            param_bounds[1] = [omega_min, omega_max]
            param_bounds[2] = [phi_min, phi_max]
            param_bounds[3] = [alpha_min, alpha_max]
    population_size : int
        Number of individuals in the population.

    Returns
    -------
    pop : 2D ndarray (float64)
        Shape (population_size, D). Each row is a chromosome with parameter values
        sampled from [low, high] for each parameter.
    """
    num_params = param_bounds.shape[0]  # D
    # Pre-allocate the population array
    pop = np.empty((population_size, num_params), dtype=np.float64)
    
    for i in range(population_size):
        for j in range(num_params):
            low = param_bounds[j, 0]
            high = param_bounds[j, 1]
            pop[i, j] = np.random.uniform(low, high)
    
    return pop

@njit
def selection(population, fitness):
    """
    Tournament selection under nopython mode.
    
    population: 2D array, shape = (num_individuals, num_params)
    fitness: 1D array, shape = (num_individuals,) with the fitness of each individual
    """
    n = population.shape[0]
    num_params = population.shape[1]

    # Pre-allocate array for the new (selected) population
    selected = np.empty((n, num_params), dtype=np.float64)

    for k in range(n):
        # Pick two random indices in [0, n)
        i = np.random.randint(0, n)
        j = np.random.randint(0, n)
        
        # Select the better chromosome
        if fitness[i] < fitness[j]:
            for col in range(num_params):
                selected[k, col] = population[i, col]
        else:
            for col in range(num_params):
                selected[k, col] = population[j, col]

    return selected


def crossover(parents, prob):
    offspring = []
    for i in range(0, len(parents), 2):
        p1, p2 = parents[i], parents[i+1]
        if np.random.rand() < prob:
            cp = np.random.randint(1, len(p1))
            child1 = np.concatenate((p1[:cp], p2[cp:]))
            child2 = np.concatenate((p2[:cp], p1[cp:]))
            offspring += [child1, child2]
        else:
            offspring += [p1, p2]
    return np.array(offspring)

@njit
def crossover(parents, prob):
    """
    Single-point crossover in nopython mode.
    
    Parameters
    ----------
    parents : 2D ndarray
        Shape (N, D), where N is the number of individuals, D is the dimension of each chromosome.
    prob : float
        Probability of crossover occurring for each pair.
        
    Returns
    -------
    offspring : 2D ndarray
        Shape (N, D). The new population after crossover.
    """
    n = parents.shape[0]
    d = parents.shape[1]
    # Allocate offspring array
    offspring = np.empty((n, d), dtype=np.float64)

    # Process pairs of parents
    for i in range(0, n, 2):
        if i + 1 >= n:
            # If there's an odd number of parents, copy the last one as is
            for col in range(d):
                offspring[i, col] = parents[i, col]
            break

        # Draw random for deciding whether to crossover
        if np.random.rand() < prob:
            # Single-point crossover index
            cp = np.random.randint(1, d)
            # Child1
            for col in range(cp):
                offspring[i, col] = parents[i, col]
            for col in range(cp, d):
                offspring[i, col] = parents[i+1, col]
            # Child2
            for col in range(cp):
                offspring[i+1, col] = parents[i+1, col]
            for col in range(cp, d):
                offspring[i+1, col] = parents[i, col]
        else:
            # No crossover -> copy parents as-is
            for col in range(d):
                offspring[i, col] = parents[i, col]
                offspring[i+1, col] = parents[i+1, col]

    return offspring

def mutate(offspring, prob, param_bounds):
    keys = list(param_bounds.keys())
    for i in range(len(offspring)):
        if np.random.rand() < prob:
            mp = np.random.randint(len(param_bounds))
            low, high = param_bounds[keys[mp]]
            offspring[i, mp] = np.random.uniform(low, high)
    return offspring

@njit
def mutate(offspring, prob, param_bounds):
    """
    Numba-friendly mutation operator.

    Parameters
    ----------
    offspring : 2D array (float64)
        Shape (N, D) where N is population size, D is number of parameters.
    prob : float
        Mutation probability in [0,1].
    param_bounds : 2D array (float64)
        Shape (D, 2). Each row is [low, high] for the corresponding parameter.

    Returns
    -------
    offspring : 2D array (float64)
        Mutated population (in-place).
    """
    n = offspring.shape[0]       # Number of individuals
    d = offspring.shape[1]       # Number of parameters

    for i in range(n):
        if np.random.rand() < prob:
            # pick a random parameter index
            mp = np.random.randint(d)
            low = param_bounds[mp, 0]
            high = param_bounds[mp, 1]
            # mutate that parameter
            offspring[i, mp] = np.random.uniform(low, high)

    return offspring

@njit
def immigration_operation(populations, fitness_values):
    # Meilleur de la pop m remplace le pire de la pop m+1
    for m in range(len(populations) - 1):
        f = fitness_values[m]
        best_idx = np.argmin(f)
        best_chrom = populations[m][best_idx]

        # Calculer fitness pour population m+1
        f_next = fitness_values[m+1]
        worst_idx = np.argmax(f_next)
        populations[m+1][worst_idx] = best_chrom
    return populations

@njit
def RSS(chromosome, data):

    y = data[:,1]
    t = data[:,0]
    t_c, alpha, omega, phi = chromosome

    f = (t_c - t) ** alpha
    g = f * np.cos(omega * np.log(t_c - t) + phi)

    V = np.column_stack((np.ones_like(f), f, g))
    try:
        A, B, C = np.linalg.inv(V.T @ V) @ (V.T @ y)
    except:
        return np.inf
    predicted = A + B * f + C * g

    return np.sum((y - predicted) ** 2)

@njit
def calculate_fitness(population, data):
    fitness = np.empty(len(population))
    for i in range(len(population)):
        fitness[i] = RSS(population[i], data)
    return fitness

def convert_param_bounds(param_bounds_dict):
    """
    Convertit un dictionnaire de bornes de paramètres en un tableau NumPy.

    Parameters
    ----------
    param_bounds_dict : dict
        Dictionnaire contenant les bornes des paramètres.

    Returns
    -------
    param_bounds_array : ndarray
        Tableau NumPy contenant les bornes des paramètres.
    """
    param_bounds_array = np.array([
        [param_bounds_dict["t_c"][0], param_bounds_dict["t_c"][1]],        # t_c
        [param_bounds_dict["omega"][0], param_bounds_dict["omega"][1]],    # omega
        [param_bounds_dict["phi"][0], param_bounds_dict["phi"][1]],        # phi
        [param_bounds_dict["alpha"][0], param_bounds_dict["alpha"][1]]     # alpha
    ], dtype=np.float64)
    
    return param_bounds_array

In [170]:
# Exemple d'utilisation
file_path = 'WTI_Spot_Price_daily.csv'
data = load_data(file_path)

time_start = "04/01/2003"  # date de début (exemple)
time_end = "11/14/2016"    # date de fin (exemple)

sample = select_sample(data, time_start, time_end)

subintervals = generate_subintervals(sample)

num_populations = 10
population_size = 10
MaxGen = 10
StopGen = 50
selection_probability = 0.9

PARAM_BOUNDS = {
    "t_c": (0, 365*10),  # 10 ans après la fin du sample en jours
    "omega": (0, 40),
    "phi": (0, 2*np.pi),
    "alpha": (0.1, 0.9)
}

param_bounds = convert_param_bounds(PARAM_BOUNDS)

# Paramètres globaux
best_solutions = []

# Boucle sur les sous-intervalles
for (sub_start, sub_end, sub_data) in subintervals:

    # Définir le crossover et mutation probability par population
    crossover_prob = np.random.uniform(0.001, 0.05, size=num_populations)
    mutation_prob = np.random.uniform(0.001, 0.05, size=num_populations)

    populations = [initialize_population(param_bounds, population_size) for _ in range(num_populations)]

    # Calculer fitness initial
    fitness_values = []
    bestObjV = np.inf
    bestChrom = None

    for m in range(num_populations):
        fit = calculate_fitness(populations[m], sub_data)
        fitness_values.append(fit)
        # Minimum fitness pour cette pop
        local_min = np.min(fit)
        if local_min < bestObjV:
            bestObjV = local_min
            bestChrom = populations[m][np.argmin(fit)]

    # gen, gen0
    gen = 1
    gen0 = 0

    # Boucle du MPGA
    while gen0 < StopGen and gen <= MaxGen:

        # Opérations génétiques
        new_populations = []
        new_fitness_values = []

        for m in range(num_populations):
            # Sélection
            fit = fitness_values[m]
            selected = selection(populations[m], fit)
            # Crossover
            offspring = crossover(selected, crossover_prob[m])
            # Mutation
            mutated = mutate(offspring, mutation_prob[m], param_bounds)
            new_populations.append(mutated)

        # Immigration
        populations = immigration_operation(new_populations, fitness_values)

        # Recalculer fitness
        fitness_values = []
        for m in range(num_populations):
            fit = np.array([RSS(ch, sub_data) for ch in populations[m]])
            fitness_values.append(fit)

        # Trouver le meilleur global du loop courant
        newbestObjV = np.inf
        for m in range(num_populations):
            local_min = np.min(fitness_values[m])
            if local_min < newbestObjV:
                newbestObjV = local_min
                newbestChrom = populations[m][np.argmin(fitness_values[m])]


        if newbestObjV < bestObjV:
            bestObjV = newbestObjV
            bestChrom = newbestChrom
            gen0 = 0
        else:
            gen0 += 1

        gen += 1

    # Sauvegarder le résultat pour ce sous-intervalle
    best_solutions.append((sub_start, sub_end, bestObjV, bestChrom))


In [167]:
best_solutions

[(1.0,
  3427.0,
  796596.306921436,
  array([3.47233567e+03, 3.10691158e-01, 1.23332778e+00, 6.71509948e-01])),
 (123.35714285714286,
  3427.0,
  817756.7004044155,
  array([3.42730236e+03, 1.04656627e+00, 1.39090555e+00, 7.95674306e-01])),
 (245.71428571428572,
  3427.0,
  769100.1022655261,
  array([3.52045714e+03, 2.16410565e-01, 2.17046181e+00, 6.78446921e-01])),
 (368.07142857142856,
  3427.0,
  738080.7709416638,
  array([3.43222375e+03, 3.77276493e-01, 1.76264368e+00, 7.75340584e-01])),
 (490.42857142857144,
  3427.0,
  707047.913812521,
  array([3.46247489e+03, 2.26280857e-01, 2.26739787e+00, 4.92458421e-01])),
 (612.7857142857143,
  3427.0,
  744171.789805808,
  array([3.48107266e+03, 9.56723528e-01, 1.86993446e+00, 5.75772544e-01])),
 (735.1428571428571,
  3427.0,
  668419.7740884951,
  array([3.49785664e+03, 2.21992680e-01, 2.74240357e+00, 3.49613932e-01])),
 (857.5,
  3427.0,
  662238.4497136718,
  array([3.52775534e+03, 7.32105147e-02, 3.10895136e+00, 8.58976388e-01])),
 

In [87]:
chromo = [4.22341053e+03, 3.22077547e+01, 2.78047960e+00, 1.91398803e-01]

@njit
def calculate_f_g(t_c, t, alpha, omega, phi):
    """
    Calculate the vectors f_j and g_j for a given t_c, alpha, omega, and phi.
    """
    
    f = (t_c - t) ** alpha
    g = f * np.cos(omega * np.log(t_c - t) + phi)
    return f, g

@njit
def calculate_linear_parameters(t, y, t_c, alpha, omega, phi):
    """
    Calculate the linear parameters A, B, and C using the LPPL model.
    """
    # Calculate f and g
    f, g = calculate_f_g(t_c, t, alpha, omega, phi)
    
    # Construct the V matrix
    V = np.column_stack((np.ones_like(f), f, g))
        
    # Compute the normal equations
    # (V_T @ V)^(-1) @ (V_T @ y)
    params = np.linalg.inv(V.T @ V) @ (V.T @ y)  # A, B, C
    
    return params  # [A, B, C]

@njit
def predict_price(t, A, B, C, t_c, alpha, omega, phi):
    """
    Predict price using the LPPL model.
    
    Parameters:
    - t : ndarray : Time points
    - A, B, C : float : Linear parameters
    - t_c : float : Critical time
    - alpha : float : Power-law exponent
    - omega : float : Angular frequency
    - phi : float : Phase

    Returns:
    - predicted : ndarray : Predicted prices
    """
    # Calculate f and g
    f = (t_c - t) ** alpha
    g = f * np.cos(omega * np.log(t_c - t) + phi)
    
    # Predicted price using the LPPL model
    predicted = A + B * f + C * g
    
    return predicted

@njit
def RSS(chromosome, data):

    y = data[:,1]
    t = data[:,0]
    t_c, alpha, omega, phi = chromosome

    f = (t_c - t) ** alpha
    g = f * np.cos(omega * np.log(t_c - t) + phi)

    V = np.column_stack((np.ones_like(f), f, g))
    A, B, C = np.linalg.inv(V.T @ V) @ (V.T @ y)

    predicted = A + B * f + C * g

    return np.sum((y - predicted) ** 2)

y = sample[:,1]
t = sample[:,0]
#RSS(chromo, y, t)

t_c, alpha, omega, phi = chromo

#f, g = calculate_f_g(t_c, t, alpha, omega, phi)
#A, B, C = calculate_linear_parameters(t, y, t_c, alpha, omega, phi)
RSS(chromo, sample)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<intrinsic exception_match>) found for signature:
 
 >>> exception_match(none, LinAlgError)
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Intrinsic in function 'exception_match': File: numba/core/unsafe/eh.py: Line 47.
    With argument(s): '(none, LinAlgError)':
   Rejected as the implementation raised a specific error:
     UnsupportedError: Exception matching is limited to <class 'Exception'>
  raised from /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numba/core/unsafe/eh.py:55

During: resolving callee type: Function(<intrinsic exception_match>)
During: typing of call at /var/folders/vk/z0p4r5z97d99dp7mpbt9yd1w0000gn/T/ipykernel_5691/2679892259.py (73)


File "../../../../../../../var/folders/vk/z0p4r5z97d99dp7mpbt9yd1w0000gn/T/ipykernel_5691/2679892259.py", line 73:
<source missing, REPL/exec in use?>


In [74]:
sample

array([[1.000e+00, 2.948e+01],
       [2.000e+00, 2.855e+01],
       [3.000e+00, 2.905e+01],
       ...,
       [3.425e+03, 4.462e+01],
       [3.426e+03, 4.339e+01],
       [3.427e+03, 4.329e+01]])

In [88]:
low = np.array([3.427e+03 + PARAM_BOUNDS["t_c"][0], PARAM_BOUNDS["omega"][0], PARAM_BOUNDS["phi"][0], PARAM_BOUNDS["alpha"][0]], dtype=np.float64)  # [t_c_min, omega_min, phi_min, alpha_min]
high = np.array([3.427e+03 + PARAM_BOUNDS["t_c"][1], PARAM_BOUNDS["omega"][1], PARAM_BOUNDS["phi"][1], PARAM_BOUNDS["alpha"][1]], dtype=np.float64)   # [t_c_max, omega_max, phi_max, alpha_max]

populations = initialize_population(low, high, 10000)

In [146]:
import numpy as np
from numba import njit

@njit
def selection(population, fitness):
    """
    Tournament selection under nopython mode.
    
    population: 2D array, shape = (num_individuals, num_params)
    fitness: 1D array, shape = (num_individuals,) with the fitness of each individual
    """
    n = population.shape[0]
    num_params = population.shape[1]

    # Pre-allocate array for the new (selected) population
    selected = np.empty((n, num_params), dtype=np.float64)

    for k in range(n):
        # Pick two random indices in [0, n)
        i = np.random.randint(0, n)
        j = np.random.randint(0, n)
        
        # Select the better chromosome
        if fitness[i] < fitness[j]:
            for col in range(num_params):
                selected[k, col] = population[i, col]
        else:
            for col in range(num_params):
                selected[k, col] = population[j, col]

    return selected

low = np.array([3.427e+03 + PARAM_BOUNDS["t_c"][0], PARAM_BOUNDS["omega"][0], PARAM_BOUNDS["phi"][0], PARAM_BOUNDS["alpha"][0]], dtype=np.float64)  # [t_c_min, omega_min, phi_min, alpha_min]
high = np.array([3.427e+03 + PARAM_BOUNDS["t_c"][1], PARAM_BOUNDS["omega"][1], PARAM_BOUNDS["phi"][1], PARAM_BOUNDS["alpha"][1]], dtype=np.float64)   # [t_c_max, omega_max, phi_max, alpha_max]

populations = initialize_population(low, high, 100000)

fitness = np.zeros(100000)

for i in range(100000):
    fitness[i] = RSS(populations[i], sample)

selection(populations, fitness)

array([[6.41991475e+03, 1.57062643e+01, 4.32136104e+00, 1.30612784e-01],
       [3.76248664e+03, 2.04074811e+01, 4.07223407e-01, 5.54413628e-01],
       [6.36646094e+03, 1.42232755e+01, 2.77591291e-01, 1.37714232e-01],
       ...,
       [6.98550515e+03, 3.96575960e+01, 2.69431942e+00, 7.15860670e-01],
       [6.46522733e+03, 1.06499331e+01, 3.72497853e-01, 7.16053262e-01],
       [4.86186450e+03, 2.97706139e+00, 4.29459829e+00, 7.30108345e-01]])

array([[4.08706720e+03, 8.54201349e+00, 4.89964808e+00, 6.22926906e-01],
       [5.64474338e+03, 4.20807389e+00, 4.89792210e+00, 2.46060303e-01],
       [4.37605894e+03, 1.18091707e+00, 1.78710812e+00, 4.95950253e-01],
       ...,
       [6.10317939e+03, 2.57795959e+01, 1.91647600e-01, 1.52852600e-01],
       [5.11112667e+03, 2.18496125e+01, 6.16715969e+00, 2.87296927e-01],
       [4.46640410e+03, 6.81719846e+00, 6.07220839e+00, 2.76424878e-01]])

In [23]:
chromo

[4223.41053, 32.2077547, 2.7804796, 0.191398803]

In [15]:
param_bounds = {
    "t_c": (sub_end, sub_end + 365*10),  # 10 ans après la fin du sample en jours
    "omega": (0, 40),
    "phi": (0, 2*np.pi),
    "alpha": (0.1, 0.9)
}

# population_size = 10

# @jit(nopython=True)
# def initialize_population(param_bounds, population_size):
#     low = [v[0] for v in param_bounds.values()]
#     high = [v[1] for v in param_bounds.values()]
#     return np.random.uniform(low, high, size=(population_size, len(param_bounds)))


# initialize_population(param_bounds, population_size)

import numpy as np
from numba import njit

# Example bounds as arrays
low = np.array([param_bounds["t_c"][0], param_bounds["omega"][0], param_bounds["phi"][0], param_bounds["alpha"][0]], dtype=np.float64)  # [t_c_min, omega_min, phi_min, alpha_min]
high = np.array([param_bounds["t_c"][1], param_bounds["omega"][1], param_bounds["phi"][1], param_bounds["alpha"][1]], dtype=np.float64)   # [t_c_max, omega_max, phi_max, alpha_max]

population_size = 10

@njit
def initialize_population(low, high, population_size):
    """
    Initialize population in nopython mode with Numba.
    low[i], high[i] = bounds for parameter i.
    """
    num_params = low.shape[0]
    pop = np.empty((population_size, num_params), dtype=np.float64)
    for i in range(population_size):
        for j in range(num_params):
            pop[i, j] = np.random.uniform(low[j], high[j])
    return pop

# Usage
population = initialize_population(low, high, population_size)


[[4.00965087e+03 3.96137903e-01 8.80970784e-01 2.66130472e-01]
 [5.42673859e+03 3.32387922e+01 4.97321847e+00 7.08082400e-01]
 [3.61638897e+03 3.10938214e+01 5.47563174e-01 7.27385285e-01]
 [4.71131080e+03 3.91073700e+01 1.16788399e+00 4.33618285e-01]
 [3.98548641e+03 2.20837719e+01 4.92665986e+00 1.03133869e-01]
 [5.35727879e+03 1.09920981e+01 5.42762909e+00 2.66398758e-01]
 [3.93500440e+03 3.44789536e+00 3.04298195e+00 8.31684167e-01]
 [6.86282588e+03 5.50388785e-01 5.45854385e+00 7.47825228e-01]
 [5.02008639e+03 1.39498772e+01 4.06391613e+00 1.21619311e-01]
 [6.98429962e+03 1.75879526e+01 1.76280224e+00 3.78097787e-01]]


In [1]:
from Utilities import load_data, select_sample
from MPGA import MPGA
import json

# Exemple d'utilisation
file_path = 'WTI_Spot_Price_daily.csv'
data = load_data(file_path)

time_start = "04/01/2003"  # date de début (exemple)
time_end = "11/14/2016"    # date de fin (exemple)

sample = select_sample(data, time_start, time_end)

mpga = MPGA(sample, "daily")

results = mpga.fit()

# Save results to a JSON file
with open("results.json", "w") as f:
    json.dump(results, f, indent=4)

/Users/julesmourgues/Documents/Cours/Master Dauphine/Paper GQ/Predicting-Oil-Price-Turning-Points/njitFunc.py:247: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'F', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))
  fitness[i] = njit_RSS(population[i], data)


In [4]:
results

[(1.0,
  3427.0,
  1403762.8929160188,
  array([3.59612122e+03, 1.64657829e+01, 5.69418292e+00, 3.51403731e-01])),
 (123.35714285714286,
  3427.0,
  1399292.836450455,
  array([3.59708118e+03, 5.52364761e+00, 5.70946601e+00, 8.82597911e-01])),
 (245.71428571428572,
  3427.0,
  1402412.0006999203,
  array([3.49717783e+03, 2.95397457e+00, 2.09905147e+00, 1.55729220e-01])),
 (368.07142857142856,
  3427.0,
  1400659.038000246,
  array([3.54381409e+03, 1.77729970e+01, 4.48131072e+00, 8.69744364e-01])),
 (490.42857142857144,
  3427.0,
  1292687.1031890276,
  array([3.62077414e+03, 4.33269138e+00, 2.68881054e+00, 5.37879259e-01])),
 (612.7857142857143,
  3427.0,
  1392970.7845062034,
  array([3.44998562e+03, 2.62371203e+01, 3.55939324e-02, 7.90806776e-01])),
 (735.1428571428571,
  3427.0,
  1008338.4887289128,
  array([3.62038808e+03, 1.89906299e+00, 4.88239518e-01, 7.41150943e-01])),
 (857.5,
  3427.0,
  1096023.4666450436,
  array([3.61852960e+03, 2.59494380e+00, 1.94209699e-01, 2.18027954e

In [7]:
mpga.fit()

In [5]:
import numpy as np
import json
from Utilities import convert_param_bounds
from njitFunc import njit_calculate_fitness, njit_selection, njit_crossover, njit_immigration_operation, njit_mutate, njit_initialize_population, njit_RSS

class MPGA:


    def __init__(self, sample, frequency = "daily"):

        # Verifiy if the sample is a 2D array
        if len(sample.shape) != 2:
            raise ValueError("The sample must be a 2D array.")
        
        self.sample = sample

        if frequency not in ["daily", "weekly", "monthly"]:
            raise ValueError("The frequency must be one of 'daily', 'weekly', 'monthly'.")
        
        self.frequency = frequency

        with open("params.json", "r") as f:
            params = json.load(f)

        self.PARAM_BOUNDS = convert_param_bounds(params["PARAM_BOUNDS"])
        self.NUM_POPULATIONS = params["NUM_POPULATIONS"]
        self.POPULATION_SIZE = params["POPULATION_SIZE"]
        self.MAX_GEN = params["MAX_GEN"]
        self.STOP_GEN = params["STOP_GEN"]
        self.SELECTION_PROBABILITY = params["SELECTION_PROBABILITY"]

        self.generate_subintervals()

    def fit(self):

        # Paramètres globaux
        best_solutions = []

        # Boucle sur les sous-intervalles
        for (sub_start, sub_end, sub_data) in self.subintervals:

            # Définir le crossover et mutation probability par population
            crossover_prob = np.random.uniform(0.001, 0.05, size=self.NUM_POPULATIONS)
            mutation_prob = np.random.uniform(0.001, 0.05, size=self.NUM_POPULATIONS)

            populations = [self.initialize_population(self.PARAM_BOUNDS, self.POPULATION_SIZE) for _ in range(self.NUM_POPULATIONS)]

            # Calculer fitness initial
            fitness_values = []
            bestObjV = np.inf
            bestChrom = None

            for m in range(self.NUM_POPULATIONS):
                fit = self.calculate_fitness(populations[m], sub_data)
                fitness_values.append(fit)
                # Minimum fitness pour cette pop
                local_min = np.min(fit)
                if local_min < bestObjV:
                    bestObjV = local_min
                    bestChrom = populations[m][np.argmin(fit)]

            # gen, gen0
            gen = 1
            gen0 = 0

            # Boucle du MPGA
            while gen0 < self.STOP_GEN and gen <= self.MAX_GEN:

                # Opérations génétiques
                new_populations = []
                new_fitness_values = []

                for m in range(self.NUM_POPULATIONS):
                    # Sélection
                    fit = fitness_values[m]
                    selected = self.selection(populations[m], fit)
                    # Crossover
                    offspring = self.crossover(selected, crossover_prob[m])
                    # Mutation
                    mutated = self.mutate(offspring, mutation_prob[m], self.PARAM_BOUNDS)
                    new_populations.append(mutated)

                # Immigration
                populations = self.immigration_operation(new_populations, fitness_values)

                # Recalculer fitness
                fitness_values = []
                for m in range(self.NUM_POPULATIONS):
                    fit = np.array([self.RSS(ch, sub_data) for ch in populations[m]])
                    fitness_values.append(fit)

                # Trouver le meilleur global du loop courant
                newbestObjV = np.inf
                for m in range(self.NUM_POPULATIONS):
                    local_min = np.min(fitness_values[m])
                    if local_min < newbestObjV:
                        newbestObjV = local_min
                        newbestChrom = populations[m][np.argmin(fitness_values[m])]


                if newbestObjV < bestObjV:
                    bestObjV = newbestObjV
                    bestChrom = newbestChrom
                    gen0 = 0
                else:
                    gen0 += 1

                gen += 1

            # Sauvegarder le résultat pour ce sous-intervalle
            best_solutions.append((sub_start, sub_end, bestObjV, bestChrom))

    
    def generate_subintervals(self):
        """
        Génère les sous-intervalles suivant la logique du pseudo-code.
        - On calcule delta comme le max entre ((time_end - time_start)*0.75/21 jours) et 21 jours.
        - subinterval_end varie de time_end à time_end - 6 semaines (42 jours) par pas de 7 jours.
        - subinterval_start varie de time_start à time_end-(time_end-time_start)/4 par pas de delta.
        """

        time_start = self.sample[0,0]
        time_end = self.sample[-1,0]

        if self.frequency == "daily":
            freq_list = [21.0, 42.0, 7.0]
        elif self.frequency == "weekly":
            freq_list = [3.0, 6.0, 1.0]
        elif self.frequency == "monthly":
            freq_list = [0.75, 1.5, 0.25]

        three_weeks, six_weeks, one_week = freq_list
        total_days = (time_end - time_start)
        delta = max((total_days * 0.75) / three_weeks, three_weeks)

        self.subintervals = []
        # Pour simplifier, on considère data[:,0] en jours continus
        for sub_end in np.arange(time_end, time_end - six_weeks, -one_week):
            for sub_start in np.arange(time_start, time_end - total_days/4, delta):
                mask = (self.sample[:,0] >= sub_start) & (self.sample[:,0] <= sub_end)
                sub_sample = self.sample[mask]            
                if len(sub_sample) > 0:
                    self.subintervals.append((sub_start, sub_end, sub_sample))
    
    def initialize_population(self, param_bounds, population_size):
        return njit_initialize_population(param_bounds, population_size)

    def selection(self, population, fitness):
        return njit_selection(population, fitness)

    def crossover(self, parents, prob):
        return njit_crossover(parents, prob)

    def mutate(self, offspring, prob, param_bounds):
        return njit_mutate(offspring, prob, param_bounds)

    def immigration_operation(self, populations, fitness_values):
        return njit_immigration_operation(populations, fitness_values)
    
    def RSS(self, chromosome, data):
        return njit_RSS(chromosome, data)

    def calculate_fitness(self, population, data):
        return njit_calculate_fitness(population, data)

In [4]:
import json
import numpy as np

# Définir les paramètres
PARAMS = {
    "param_bounds": {
        "t_c": (0, 365*10),  # 10 ans après la fin du sample en jours
        "omega": (0, 40),
        "phi": (0, 2*np.pi),  # Utiliser np.pi directement
        "alpha": (0.1, 0.9)
    },
    "num_populations": 10,
    "population_size": 10,
    "MaxGen": 10,
    "StopGen": 50,
    "selection_probability": 0.9
}

# Convertir np.pi en une valeur numérique
#PARAMS["param_bounds"]["phi"] = (0, float(np.pi * 2))

# Enregistrer le fichier de config en JSON
with open("params.json", "w") as f:
    json.dump(PARAMS, f, indent=4)

# Charger le fichier de config
with open("params.json", "r") as f:
    params = json.load(f)

# Afficher les paramètres chargés
print(params)

{'param_bounds': {'t_c': [0, 3650], 'omega': [0, 40], 'phi': [0, 6.283185307179586], 'alpha': [0.1, 0.9]}, 'num_populations': 10, 'population_size': 10, 'MaxGen': 10, 'StopGen': 50, 'selection_probability': 0.9}
